In [1]:
import sys, os
import pandas as pd
import dask
import dask.dataframe as dd
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import KFold ,RandomizedSearchCV ,train_test_split, cross_val_predict , cross_val_score
# from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,classification_report, confusion_matrix,accuracy_score,matthews_corrcoef
# from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
# import matplotlib.pyplot as plt
# # import seaborn as sns
# import cesium as cs
# from cesium import featurize
import numpy as np
np.random.seed(seed = 101)
# import xgboost as xgb; print('XGBoost Version:', xgb.__version__)

In [2]:
# !pip install cesium

In [3]:
#the funtions will tak a dataframe 
# get the repeated timepoints, 
# divide the data frame 
# average all the columnns repeated 
# return a new dataframe 
def average_column_selected (dataframe,cell_line):
    dataframe = dataframe.drop("treatment",axis=1)
    dataframe = dataframe [ dataframe["cell_line"] == cell_line   ]
    detection_df = dataframe.groupby("time").nunique()
    reapeated_lines = detection_df[detection_df["GAPDH"] > 1].index.values.tolist()
    
#     dataframe["cell_line"] 
    
    df_clean_part_one = dataframe[ ~dataframe["time"].isin(reapeated_lines)  ]
    
    average_df_2 = pd.DataFrame()
    for m in reapeated_lines:
#         print ( dataframe[dataframe["time"]==m])
        average_df = dataframe [dataframe["time"]==m].mean()
#         print (type(average_df))
#         average_df = pd.concat([ average_df, dataframe[dataframe["time"]==m] ].mean()   ,axis=0)
#         print (pd.DataFrame(average_df).T)
        df_temp2 = pd.DataFrame(average_df).T
        average_df_2 = pd.concat([average_df_2,df_temp2], axis=0,sort=False)
#         print (pd.DataFrame(average_df))


    new_values = average_df_2
#     print (new_values)
    
    new_dataframe = pd.concat([ df_clean_part_one, new_values ],axis=0,sort=False)
    new_dataframe["cell_line"]= cell_line
    return new_dataframe
    

In [99]:
my_colums_to_gene_names= { 
'b.CATENIN':'CTNNB1',
'cleavedCas':'BCAR1',
 'CyclinB':'CCNB1',
 'GAPDH':'GAPDH',
 'Ki.67': 'MKI67',
 'p.4EBP1':'EIF4EBP1',
 'p.Akt.Ser473.':'AKT1',
 'p.AKT.Thr308.':'AKT1',
 'p.AMPK':'PRKAB1',
 'p.BTK':'BTK',
 'p.CREB':'CREB1',
 'p.ERK':'MAPK3',
'p.FAK':'PTK2', 
 'p.GSK3b':'GSK3B', 
 'p.H3':'H3F3A', 
 'p.HER2':'ERBB2', 
 'p.JNK':'MAPK8', 
 'p.MAP2K3':'MAP2K3', 
 'p.MAPKAPK2':'MAPKAPK2',
'p.MEK':'MAP2K1', 
 'p.MKK3.MKK6':'MAP2K3', ## given taht in the network MAP2K3 has bigger conectivity I preserved this 
#  'p.MKK3.MKK6b':'MAP2K6', ### I guess both are interfacing and phospholirated
 'p.MKK4':'MAP2K4', 
 'p.NFkB':'NFKB1', 
 'p.p38':'MAPK1', 
 'p.p53':'TP53',
'p.p90RSK':'RPS6KA1', 
 'p.PDPK1':'PDPK1', 
 'p.PLCg2':'PLCG2', 
 'p.RB':'RB1', 
 'p.S6':'RPS6', 
 'p.S6K':'RPS6KB1', 
 'p.SMAD23':'SMAD2',
    'p.SRC':'SRC', 
 'p.STAT1':'STAT1', 
 'p.STAT3':'STAT3', 
 'p.STAT5':'STAT5A'
}

In [5]:
big_df = pd.DataFrame()
all_files = [x for x in os.listdir("synapse_DREAM_challenge/CELL_LINES/") if ".csv" in x and x != "subchallenge_1_template_data.csv"  ]

#target cell lines subch one
target_cell_lines = ['AU565', 'EFM19', 'HCC2218', 'LY2', 'MACLS2', 'MDAMB436']

#targert genes subchallemnge one
target_genes = ['p.ERK', 'p.Akt.Ser473.', 'p.S6', 'p.HER2', 'p.PLCg2']


for m in all_files:
    #print (m)
    df_temp = dd.read_csv("synapse_DREAM_challenge/CELL_LINES/%s"%m)
    big_df = dd.concat([big_df,df_temp],axis=0)

In [6]:
# for c in big_df.columns:
#     print("{}".format( c.split(",")[-1] ) )

In [7]:
train_df = big_df[~big_df["cell_line"].isin(target_cell_lines)]

train_df = train_df[train_df["treatment"]!="full"]

In [8]:
train_df=train_df.compute()

In [9]:
my_new_df_average = pd.DataFrame()
for cell_line in train_df["cell_line"].unique():
    for m in train_df["treatment"].unique():
        for n in  train_df["cellID"].unique()[:2]:
    #         slide_df = X[ (  X["treatment"]== "iMEK")  & ( X["cellID"]==2 ) ]
            slide_df = train_df[ (  train_df["treatment"]== m)  & ( train_df["cellID"]==n ) ]
            average_df = average_column_selected( slide_df , cell_line )
            average_df["treatment"] = m
            
#             print ( slide_df["cell_line"] )
    #         average_df["cell_line"] = slide_df["cell_line"].unique() 
    #         print ( slide_df["cell_line"] )
    #         print (slide_df["cell_line"].unique())

            my_new_df_average = pd.concat([my_new_df_average,average_df],axis=0,sort=False)

#             print (m )
    #         print ( slide_df )


In [10]:
X = my_new_df_average.drop(target_genes,axis=1)

y = my_new_df_average[target_genes]

In [11]:
print ("get dummies")
my_dummies = pd.get_dummies(X["treatment"])

get dummies


In [12]:
print ("filling NA")
# y = y.compute()

for m in y.columns :
    y[m].fillna(y[m].mean() , inplace=True )
# X.fillna(0)
# y.fillna(0)

filling NA


/home/didier/anaconda3/envs/dask-tutorial/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [13]:
### load proteomics
df_proteomics = pd.read_csv('/home/didier/Documents/BARRADD_Things/DREAM_challenge/kaust_rss/Proteomics_log2FC_usefuldata_genemat.csv')

In [14]:
### load marcotte
df_cnv = pd.read_csv('../Cancer_marker_Challenge/CNV_Marcotte.csv')

In [107]:
df_cnv=df_cnv.rename(columns = {'Unnamed: 0':'Cellline'})

In [15]:
df_proteomics.head()

,Cellline,NUDT4B,CYP2D7,CYP2D6,SMIM26,TRAV16,GATD3B,GATD3A,MMP24OS,RBM47,...,FAP,MAN2A2,SESN1,SLC2A3,SLC2A14,LTO1,R3HCC1L,SLC37A2,BTBD10,MAPK8IP3
0,AU565,2.280948,0.534789,0.534789,0.666483,1.217341,1.269833,1.269833,0.985800,7.289513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BT20,-0.336647,0.214997,0.214997,-1.476875,NaN,0.410916,0.410916,NaN,6.718455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BT474,2.289154,0.147820,0.147820,-0.998707,0.729033,1.330236,1.330236,1.936845,6.772977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BT483,-1.238482,-0.376452,-0.376452,NaN,0.828461,0.180556,0.180556,-1.032499,4.743943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BT549,-0.288732,0.093583,0.093583,NaN,2.267953,-0.481540,-0.481540,-0.065824,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
for x in df_cnv.columns:
#     [a.k.a. PRKM1, ERK2, PRKM2, ERK]
    if "H3F3" in x or "MAP2K1" in x or "STAT5" in x:
        print (x)

H3F3A
H3F3AP1
H3F3AP3
H3F3AP4
H3F3AP5
H3F3AP6
H3F3B
H3F3BP1
H3F3BP2
H3F3C
MAP2K1
MAP2K1P1
STAT5A
STAT5B


In [110]:
##cheack all names are in te corresponding name 
my_found = []
for col,gene in my_colums_to_gene_names.items():
#     for m in df_cnv.columns:
    for m in df_proteomics.columns:
#         if "RAC" in gene:
        if gene in m :
#             print (m)
            if gene not in my_found:

                my_found.append(gene)
                
#             print (gene)
          
#     print (col,gene,df_cnv[gene].shape)
    print ( gene)
 

CTNNB1
BCAR1
CCNB1
GAPDH
MKI67
EIF4EBP1
AKT1
AKT1
PRKAB1
BTK
CREB1
MAPK3
PTK2
GSK3B
H3F3A
ERBB2
MAPK8
MAP2K3
MAPKAPK2
MAP2K1
MAP2K3
MAP2K4
NFKB1
MAPK1
TP53
RPS6KA1
PDPK1
PLCG2
RB1
RPS6
RPS6KB1
SMAD2
SRC
STAT1
STAT3
STAT5A


In [77]:
# X["time"] = X["time"].astype('datetime64[m]')
for col,gene in my_colums_to_gene_names.items():
    if gene not in my_found:
        print (gene)
# (my_found)

In [79]:
# X["time"] =pd.to_datetime(X['time'])
# my_colums_to_gene_names

In [105]:
# my_found

In [ ]:
# my_columns = ['b.CATENIN', 'cleavedCas',
#        'CyclinB', 'GAPDH', 'IdU', 'Ki.67', 'p.4EBP1', 'p.AKT.Thr308.',
#        'p.AMPK', 'p.BTK', 'p.CREB', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.JNK',
#        'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB',
#        'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.RB', 'p.S6K', 'p.SMAD23',
#        'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5']

In [109]:
df_cnv.columns

Index(['Cellline', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1',
       'A2ML1-AS1', 'A2MP1', 'A3GALT2',
       ...
       'ZNF177', 'ZNF223', 'ZNF286A', 'ZNF33B', 'ZNF668', 'ZNF709', 'ZNF747',
       'ZNF763', 'ZNF788', 'ZNF8'],
      dtype='object', length=29125)

In [106]:
df_proteomics.head()

,Cellline,NUDT4B,CYP2D7,CYP2D6,SMIM26,TRAV16,GATD3B,GATD3A,MMP24OS,RBM47,...,FAP,MAN2A2,SESN1,SLC2A3,SLC2A14,LTO1,R3HCC1L,SLC37A2,BTBD10,MAPK8IP3
0,AU565,2.280948,0.534789,0.534789,0.666483,1.217341,1.269833,1.269833,0.985800,7.289513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BT20,-0.336647,0.214997,0.214997,-1.476875,NaN,0.410916,0.410916,NaN,6.718455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BT474,2.289154,0.147820,0.147820,-0.998707,0.729033,1.330236,1.330236,1.936845,6.772977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BT483,-1.238482,-0.376452,-0.376452,NaN,0.828461,0.180556,0.180556,-1.032499,4.743943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BT549,-0.288732,0.093583,0.093583,NaN,2.267953,-0.481540,-0.481540,-0.065824,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
